In [281]:
import os
import cv2
import numpy as np
import tensorflow as tf
import sys

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

# Import utilites
from utils import label_map_util
from utils import visualization_utils as vis_util

In [282]:
# Name of the directory containing the object detection module we're using
MODEL_NAME = 'inference_graph'
IMAGE_NAME = 'sort-9.jpg'

# Grab path to current working directory
CWD_PATH = os.getcwd()

# Path to frozen detection graph .pb file, which contains the model that is used
# for object detection.
PATH_TO_CKPT = os.path.join(CWD_PATH,MODEL_NAME,'frozen_inference_graph.pb')

# Path to label map file
PATH_TO_LABELS = os.path.join(CWD_PATH,'training','labelmap.pbtxt')

# Path to image
PATH_TO_IMAGE = os.path.join(CWD_PATH,IMAGE_NAME)

# Number of classes the object detector can identify
NUM_CLASSES = 3

# Load the label map.
# Label maps map indices to category names, so that when our convolution
# network predicts `5`, we know that this corresponds to `king`.
# Here we use internal utility functions, but anything that returns a
# dictionary mapping integers to appropriate string labels would be fine
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [283]:
# Load the Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

    sess = tf.Session(graph=detection_graph)

In [284]:
# Define input and output tensors (i.e. data) for the object detection classifier

# Input tensor is the image
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

# Output tensors are the detection boxes, scores, and classes
# Each box represents a part of the image where a particular object was detected
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

# Each score represents level of confidence for each of the objects.
# The score is shown on the result image, together with the class label.
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')

# Number of objects detected
num_detections = detection_graph.get_tensor_by_name('num_detections:0')

# Load image using OpenCV and
# expand image dimensions to have shape: [1, None, None, 3]
# i.e. a single-column array, where each item in the column has the pixel RGB value
image = cv2.imread(PATH_TO_IMAGE)
image_expanded = np.expand_dims(image, axis=0)

# Perform the actual detection by running the model with the image as input
(boxes, scores, classes, num) = sess.run(
    [detection_boxes, detection_scores, detection_classes, num_detections],
    feed_dict={image_tensor: image_expanded})

In [285]:
objects = []
for index, value in enumerate(classes[0]):
    object_dict = {}
#     print(index,value)
#     print(category_index.get(value).get('name'))
    if scores[0, index] > 0.50:

        print(category_index.get(value).get('name'))

BG
RH
LH


In [286]:
category_index.get(value)

{'id': 1, 'name': 'RH'}

In [287]:
objects

[]

In [288]:
frame = image

objects = []
for index, value in enumerate(classes[0]):
    object_dict = {}
#     print(index,value)
#     print(category_index.get(value).get('name'))
    if scores[0, index] > 0.60:

        print(category_index.get(value).get('name'))
        objects.append(category_index.get(value).get('name'))
        print(objects)
#         print(category_index.get(value).get('name'))
height, width, c = frame.shape
for i,j in zip(range(len(boxes)+2),objects):
    print(i)
    object_dict = {}
    if boxes[0][i][0]>0 and boxes[0][i][1]>0:
#         print(category_index.get(value).get('name'))
        if scores[0][i] > 0.60:
            ymin = int(boxes[0][i][0]*height)
            xmin = int(boxes[0][i][1]*width )
            ymax = int(boxes[0][i][2]*height)
            xmax = int(boxes[0][i][3]*width )       
            x1 = xmin
            x2 = xmax
            y1 = ymin
            y2 = ymax
#             print(xmin, ymin, xmax, ymax)
            center_x = int((xmin+xmax)/2)

            center_y = int((ymax+ymin)/2)

            center = (center_x,center_y)
            
            
                #pts.appendleft(center)
                # print(center)
            thickness = 2
#             print(center)
            start_point =(xmin, ymin)
            end_point = (xmax, ymax)
            circle  = cv2.circle(frame, center, 5, (0, 0, 255), -1)
            color = (255, 0, 0)
            rect = cv2.rectangle(image, start_point, end_point, color, thickness)
            
            rect = cv2.putText(rect, str(j), (xmin, ymin-10), cv2.FONT_HERSHEY_SIMPLEX, 2.0, (0,0,255), 5)
#             cv2.imwrite('crop'+str(i)+'.jpg',frame[center_x-150:center_x+150+x2,center_y-150:center_y+150])
            if j=='BG':
                bxmin,bymin,bxmax,bymax = xmin, ymin, xmax, ymax
                
            if j=='RH':
                rxmin,rymin,rxmax,rymax = xmin, ymin, xmax, ymax
                r_center = center
            if j=='LH':
                lxmin,lymin,lxmax,lymax = xmin, ymin, xmax, ymax
                l_center = center
                
            detected_rect = [xmin, ymin, xmax, ymax]
            print(j,detected_rect)
# print(center)

# cv2.imwrite('circle.jpg',circle)
cv2.imwrite('rect.jpg',rect)



BG
['BG']
RH
['BG', 'RH']
LH
['BG', 'RH', 'LH']
0
BG [546, 252, 2473, 2271]
1
RH [1578, 1075, 2186, 1734]
2
LH [696, 867, 1354, 1528]


True

In [274]:
bxmin,bymin,bxmax,bymax

(546, 252, 2473, 2271)

In [278]:
pl = 160/(bxmax-bxmin)
pl

0.08303061754021795

# distance between staring point and RH center



In [279]:
start_x,start_y = bxmin,bymin



NameError: name 'start' is not defined

In [270]:
for i in range(len(boxes)):
    print("i = ",i)
    if boxes[0][i][0]>0 and boxes[0][i][1]>0:
        if scores[0][i] > 0.90:
            ymin = (boxes[0][i][0])#*height)
            xmin = (boxes[0][i][1])#*width )
            ymax = (boxes[0][i][2])#*height)
            xmax = (boxes[0][i][3])#*width )       
            print(xmin, ymin, xmax, ymax)

i =  0
0.16756861 0.10299464 0.7578495 0.92771953


In [167]:
im = cv2.imread(PATH_TO_IMAGE)
plt.imshow(im)
plt.show()

/home/gokul/anaconda3/envs/tensorflow_frcnn/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  This is separate from the ipykernel package so we can avoid doing imports until


In [168]:

boxes[0][20][3]

0.7086005

In [169]:
height, width, c 

(480, 640, 3)

In [170]:
detected_rect

[145, 86, 462, 404]

In [171]:
import matplotlib.pyplot as plt
# import utils
plt.imshow(image)
plt.show()

/home/gokul/anaconda3/envs/tensorflow_frcnn/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  after removing the cwd from sys.path.


In [172]:
objects = []
for index, value in enumerate(classes[0]):
    object_dict = {}
    if scores[0, index] > 0.90:

        print(category_index.get(value).get('name'))
        


BG


In [173]:
vis_util.visualize_boxes_and_labels_on_image_array(
    image,
    np.squeeze(boxes),
    np.squeeze(classes).astype(np.int32),
    np.squeeze(scores),
    category_index,
    use_normalized_coordinates=True,
    line_thickness=3,
    min_score_thresh=0.90)

# All the results have been drawn on image. Now display the image.
# cv2.imshow('Object detector', image)
print(image.size)
print(len(boxes))


# Press any key to close the image
cv2.waitKey(0)

# Clean up
cv2.destroyAllWindows()


921600
1


In [174]:
boxes.shape

(1, 300, 4)

In [175]:
scores>0.9

array([[ True, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
      